<a href="https://colab.research.google.com/github/DB9653/Bachelorarbeit/blob/main/HASOCOne_(TrainH19_TestG18).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA

In [ ]:
## https://github.com/suman101112/hasoc-fire-2020/blob/main/HASOCOne_team_german_task_a.ipynb

import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")



In [ ]:
device


device(type='cuda')

# DATA

In [ ]:
import pandas as pd
import math
from sklearn import preprocessing
import csv

#file = "hasoc_2020_de_train_new_a.xlsx"
#file_test = "german_test_1509.csv"

# df_train = pd.read_csv("/content/german_dataset2019.tsv",sep="\t", converters={'tweet_id':int})

df_train = pd.read_csv("/content/german_dataset2019.tsv",sep="\t")
#df_train = df_train.dropna()

df_test = pd.read_csv("/content/germeval2018test.txt",sep='\t', encoding = "UTF-8", quoting=csv.QUOTE_NONE, names=("text", "task1", "task2"))

df_test['task1'] = df_test['task1'].str.replace('OTHER', 'NOT')
df_test
df_test['task1'] = df_test['task1'].str.replace('OFFENSE', 'HOF')
df_test



task = 'task1'
task_2019 = 'task_1'

#2019 datasets also


#file_2019_1 = pd.read_csv("2019/german_dataset/german_dataset/hasoc_de_test_gold.tsv",sep='\t')
#file_2019_2 = pd.read_csv("2019/german_dataset/german_dataset/german_dataset.tsv",sep="\t") 

#test
#file_2019_1 = pd.read_csv("/content/hasoc_de_test_gold.tsv",sep='\t', converters={'tweet_id':int})
#file_2020_train = pd.read_excel("/content/hasoc_2020_de_train_new_a.xlsx", converters={'tweet_id':int})

#train
#file_2019_2 = pd.read_csv("/content/german_dataset2019.tsv",sep="\t", converters={'tweet_id':int})
#file_2020_test = pd.read_csv("/content/german_test_1509.csv", converters={'tweet_id':int})


#sentences_2019_1 = list(file_2019_1['text'].values)
#sentences_2019_2 = list(file_2019_2['text'].values)


#sentences_2020_train = list(file_2020_train['text'].values)
#sentences_2020_test = list(file_2020_test['text'].values)

#labels_2019_1 = list(file_2019_1[task_2019].values)
#labels_2019_2 = list(file_2019_2[task_2019].values)

#labels_2020_train = list(file_2020_train['task1'].values)
#labels_2020_test = list(file_2020_test['task1'].values)

print(f"Len df_train: {len(df_train)}")
display(df_train)



Len df_train: 3819


,text_id,text,task_1,task_2
0,hasoc_de_1,Frank Rennicke – Ich bin stolz https://t.co/Cm...,NOT,NONE
1,hasoc_de_2,ANSEHEN.....und danach bitte TEILEN...TEILEN.....,NOT,NONE
2,hasoc_de_3,#Koeln Mohamed erkennt kein deutsches Recht so...,NOT,NONE
3,hasoc_de_4,#SaudiArabien ist eine brutale islamische Dikt...,NOT,NONE
4,hasoc_de_5,Bundespolizei #München hat im 1. Quartal 2019 ...,NOT,NONE
...,...,...,...,...
3814,hasoc_de_3815,"akquirieren Männer,die sich um die Kinder kümm...",NOT,NONE
3815,hasoc_de_3816,"Ja,schon seit Jahren! Ich muß dran denken,das...",NOT,NONE
3816,hasoc_de_3817,@Kurkamp @wendt_joachim @Schroeder_Live @lamni...,NOT,NONE
3817,hasoc_de_3818,EU Wahlen! AfD für Kindererziehungszeiten für...,NOT,NONE


In [ ]:
display(df_test)

,text,task1,task2
0,"Meine Mutter hat mir erzählt, dass mein Vater ...",NOT,OTHER
1,@Tom174_ @davidbest95 Meine Reaktion; |LBR| Ni...,NOT,OTHER
2,"#Merkel rollt dem Emir von #Katar, der islamis...",NOT,OTHER
3,„Merle ist kein junges unschuldiges Mädchen“ K...,NOT,OTHER
4,@umweltundaktiv Asylantenflut bringt eben nur ...,HOF,ABUSE
...,...,...,...
3527,"@schlabonski tja, es können einen nicht alle L...",NOT,OTHER
3528,"@RKnillmann @lawyerberlin @AfD Aha, der Islam ...",HOF,ABUSE
3529,"@podilein Mannheim, weltoffen und kunterbunt. ...",HOF,ABUSE
3530,@stephanweil was ist nun mit kostenlosen Kitas...,NOT,OTHER


In [ ]:
#file_2020_test

In [ ]:
print(df_train.dtypes)

text_id    object
text       object
task_1     object
task_2     object
dtype: object


In [ ]:
labelsss = df_train['task_1']
print(labelsss.value_counts())

NOT    3412
HOF     407
Name: task_1, dtype: int64


In [ ]:
total_sentences = list(df_train['text'].values)
total_labels = list(df_train['task_1'].values)

#total_sentences.extend(sentences_2019_1)
#total_sentences.extend(sentences_2019_2)

#total_sentences.extend(sentences_2020_train)
#total_sentences.extend(sentences_2020_test)

#total_labels.extend(labels_2019_1)
#total_labels.extend(labels_2019_2)

#total_labels.extend(labels_2020_train)
#total_labels.extend(labels_2020_test)

test_sentences = list(df_test['text'].values)
test_labels = list(df_test['task1'].values)

# Cleaning, Transforming, Encoding

In [ ]:
def clean_text(sentences):
    for index,line in enumerate(sentences):
        if "\n" in line:
            sentences[index] = line.replace("\n","")
    return sentences
        
total_sentences = clean_text(total_sentences)
test_sentences = clean_text(test_sentences)

def clean_labels(labels):
    new_list= []
    for value in labels:
        new_list.append(value.strip())
    return new_list

total_labels = clean_labels(total_labels)
test_labels = clean_labels(test_labels)

le = preprocessing.LabelEncoder()
le.fit(total_labels)
encoded_labels = le.transform(total_labels)
encoded_test_labels = le.transform(test_labels)
print("encoded labels:", set(encoded_labels))

print(f"\nLen.: total_sentences: {len(total_sentences)}, encoded labels: {len(encoded_labels)}, test_sentences: {len(test_sentences)}, encoded_test_labels: {len(encoded_test_labels)}")

display(df_test)



encoded labels: {0, 1}

Len.: total_sentences: 3819, encoded labels: 3819, test_sentences: 3532, encoded_test_labels: 3532


,text,task1,task2
0,"Meine Mutter hat mir erzählt, dass mein Vater ...",NOT,OTHER
1,@Tom174_ @davidbest95 Meine Reaktion; |LBR| Ni...,NOT,OTHER
2,"#Merkel rollt dem Emir von #Katar, der islamis...",NOT,OTHER
3,„Merle ist kein junges unschuldiges Mädchen“ K...,NOT,OTHER
4,@umweltundaktiv Asylantenflut bringt eben nur ...,HOF,ABUSE
...,...,...,...
3527,"@schlabonski tja, es können einen nicht alle L...",NOT,OTHER
3528,"@RKnillmann @lawyerberlin @AfD Aha, der Islam ...",HOF,ABUSE
3529,"@podilein Mannheim, weltoffen und kunterbunt. ...",HOF,ABUSE
3530,@stephanweil was ist nun mit kostenlosen Kitas...,NOT,OTHER


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 28.3 MB/s 
     |████████████████████████████████| 6.6 MB 55.4 MB/s 
     |████████████████████████████████| 120 kB 76.2 MB/s 


In [ ]:
from transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

max_length = 0
for sentence in total_sentences:
    #print(sentence)
    length = len(tokenizer.tokenize(sentence))
    if length > max_length:
        max_length  = length
print("max token length is: ",max_length)
# max token length obtained is 50
# bert tokens are limited to 514 bytes.



max token length is:  162


In [ ]:
def encoder_generator(sentences,labels):
    
    sent_index = []
    input_ids = []
    attention_masks =[]

    for index,sent in enumerate(sentences):
        
        sent_index.append(index)
        
        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=max_length,

                                             #deprecation, padding geändert
                                             padding='max_length',

                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids,dim=0)
    attention_masks = torch.cat(attention_masks,dim=0)
    labels = torch.tensor(labels)
    sent_index = torch.tensor(sent_index)

    return sent_index,input_ids,attention_masks,labels

sent_index,input_ids,attention_masks,encoded_label_tensors = encoder_generator(total_sentences,encoded_labels)
test_sent_index,test_input_ids,test_attention_masks,encoded_test_label_tensors = encoder_generator(test_sentences,encoded_test_labels)
print('Original: ', total_sentences[0])
print('Token IDs:', input_ids[0])
#print(encoded_label_tensors)
#print(encoded_test_label_tensors)

Original:  Frank Rennicke – Ich bin stolz https://t.co/Cm6TD8w1k1 https://t.co/qynXso07Zn
Token IDs: tensor([  101, 11872, 52712, 71609, 10112,   100, 21023, 16292, 47264, 48275,
        14120,   131,   120,   120,   188,   119, 11170,   120,   140, 10147,
        11211, 95936, 11396, 10874, 10759, 10174, 10759, 14120,   131,   120,
          120,   188,   119, 11170,   120,   185, 13379, 12674, 11669, 10929,
        11305, 13966, 10115,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0

# Split Data into Train, Val and Test Data

In [ ]:
from torch.utils.data import TensorDataset,random_split

dataset = TensorDataset(input_ids,attention_masks,encoded_label_tensors)
test_dataset = TensorDataset(test_sent_index,test_input_ids,test_attention_masks,encoded_test_label_tensors)

train_size = int(0.75*len(dataset))

val_size = len(dataset)-train_size

train_dataset,val_dataset = random_split(dataset,[train_size,val_size])

print('train data samples is {}'.format(len(train_dataset)))
print("valid data samples is {}".format(len(val_dataset)))
print("test data samples is {}".format(len(test_dataset)))

train data samples is 2864
valid data samples is 955
test data samples is 3532


In [ ]:
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=8

train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(val_dataset,
                              sampler=SequentialSampler(val_dataset),
                              batch_size=bs)
test_data_loader = DataLoader(test_dataset,
                            sampler=SequentialSampler(test_dataset),
                            batch_size=bs)



# MODEL

In [ ]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',
                                                     num_labels=len(le.classes_),
                                                     output_attentions=False,
                                                     output_hidden_states=False,
                                                     )
model.cuda()
#model.cpu()



Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# deprecation, AdamW zu torch.optim.AdamW
optimizer = torch.optim.AdamW(model.parameters(),lr=2e-5,eps=1e-8)

from transformers import get_linear_schedule_with_warmup

epochs=10
total_steps = len(train_data_loader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps=0,
                                           num_training_steps=total_steps)



In [ ]:
import numpy as np

def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label



In [ ]:
import random
import numpy as np
import time
from sklearn.metrics import classification_report,accuracy_score,f1_score

total_t0 = time.time()

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)



In [ ]:
torch.FloatTensor.device

<attribute 'device' of 'torch._C._TensorBase' objects>

# Train and evaluate model

In [ ]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    #return correct.sum() / torch.FloatTensor([y.shape[0]]).cuda()-1

    return correct.sum() / torch.FloatTensor([y.shape[0]]).cuda()
    #return correct.sum() 
def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label



In [ ]:
#print(torch.FloatTensor.parameters().device)
print(torch.FloatTensor.device)

print(next(model.parameters()).device)

<attribute 'device' of 'torch._C._TensorBase' objects>
cuda:0


In [ ]:
def train():
    total_train_loss = 0
    total_train_acc = 0
    
    model.train() # set model in train mode for batchnorm and dropout layers in bert model
    
    for step,batch in enumerate(train_data_loader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
            
        #loss,logits = model(b_input_ids,
        #                    attention_mask = b_input_mask,
        #                    labels = b_labels.long()).values() 
                            #return_dict = False
                            #).values()


        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = output[0]
        logits = output[1]
            
        total_train_loss += loss.item()
        total_train_acc += categorical_accuracy(logits,b_labels).item()
            
        loss.backward()
            
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
            
        optimizer.step()
            
        scheduler.step() #go ahead and update the learning rate
            
    avg_train_loss = total_train_loss/len(train_data_loader)
    avg_train_acc = total_train_acc/len(train_data_loader)
    
    return avg_train_loss,avg_train_acc

In [ ]:
def evaluate():
    model.eval()
        
    total_eval_accuracy = 0
    total_eval_loss = 0
    number_of_eval_steps= 0
    
    all_true_labels = []
    all_pred_labels = []

    for batch in valid_data_loader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():

            loss, logits = model(b_input_ids,
                                attention_mask= b_input_mask,
                                labels = b_labels.long()).values()
                                #return_dict = False
                                #).values()
            
            #answer_start_scores, answer_end_scores = model(**inputs).values()

            #outputs = model(**inputs)
            #answer_start_scores = outputs.start_logits
            #answer_end_scores = outputs.end_logits

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()

        label_ids = b_labels.to('cpu').numpy()

        pred,true = predictions_labels(logits,label_ids)
        
        all_pred_labels.extend(pred)
        all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    macro_f1_score = f1_score(all_pred_labels,all_true_labels,average='macro')
    
    avg_val_loss = total_eval_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))
    
    return avg_val_loss,avg_val_accuracy,macro_f1_score

In [ ]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
epochs = 7

best_macro_f1 = float('0')

for epoch in range(epochs):
    
    start_time = time.time()
    train_loss,train_acc = train()
    valid_loss,valid_acc,macro_f1 = evaluate()
    
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        torch.save(model,'model_german_task_a.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.90      0.95       955

    accuracy                           0.90       955
   macro avg       0.50      0.45      0.47       955
weighted avg       1.00      0.90      0.95       955

accuracy = 0.90
Epoch: 01 | Epoch Time: 1m 57s
	Train Loss: 0.367 | Train Acc: 88.97%
	 Val. Loss: 0.321 |  Val. Acc: 89.84%


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.90      0.95       955

    accuracy                           0.90       955
   macro avg       0.50      0.45      0.47       955
weighted avg       1.00      0.90      0.95       955

accuracy = 0.90
Epoch: 02 | Epoch Time: 1m 56s
	Train Loss: 0.355 | Train Acc: 89.18%
	 Val. Loss: 0.321 |  Val. Acc: 89.84%
              precision    recall  f1-score   support

           0       0.04      0.50      0.08         8
           1       1.00      0.90      0.95       947

    accuracy                           0.90       955
   macro avg       0.52      0.70      0.51       955
weighted avg       0.99      0.90      0.94       955

accuracy = 0.90
Epoch: 03 | Epoch Time: 1m 57s
	Train Loss: 0.314 | Train Acc: 88.62%
	 Val. Loss: 0.313 |  Val. Acc: 89.84%
              precision    recall  f1-score   support

           0       0.13      0.46      0.

In [ ]:
del model
import gc
gc.collect()





357

# Load trained model and evaluate on test data

In [ ]:
#model = torch.load('HASOCOne_HASOC2019_model_german_task_a.pt')
model = torch.load('model_german_task_a.pt')

model = model.to(device)

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#model = TheModelClass(*args, **kwargs)

#model = torch.load(/content/HASOCOne_HASOC2019_model_german_task_a.pt)
#model.load_state_dict(torch.load("/content/HASOCOne_HASOC2019_model_german_task_a.pt"))


# Name nochmal checken!!!
#model = torch.load("/content/HASOCOne_HASOC2019_model_german_task_a.pt")



In [ ]:
#path = "/content/drive/MyDrive/Colab Notebooks/{HASOCOne_HASOC2019_model_german_task_a.pt}" 
#torch.save(model.state_dict(), path)

In [ ]:
#del model
#import gc
#gc.collect()

#model = torch.load('model_german_task_a.pt')
#model = model.to(device)

def evaluate_test():
    model.eval()
        
    total_eval_accuracy = 0
    total_eval_loss = 0
    number_of_eval_steps= 0
    
    all_true_labels = []
    all_pred_labels = []
    
    all_sentence_id=[]

    for batch in test_data_loader:
        b_sentence_id = batch[0].to(device)
        b_input_ids = batch[1].to(device)
        b_input_mask = batch[2].to(device)
        b_labels = batch[3].to(device)

        sent_ids = b_sentence_id.to('cpu').numpy()
        all_sentence_id.extend(sent_ids)
        
        with torch.no_grad():

            loss, logits = model(b_input_ids,
                                attention_mask= b_input_mask,
                                labels = b_labels.long(),
                                return_dict = False)
        total_eval_loss+=loss.item()

        logits = logits.detach().cpu().numpy()

        label_ids = b_labels.to('cpu').numpy()
        

        pred,true = predictions_labels(logits,label_ids)
        
        all_pred_labels.extend(pred)
        
        all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    
    avg_val_loss = total_eval_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))
    
    return avg_val_loss,avg_val_accuracy,all_sentence_id,all_pred_labels

valid_loss,valid_acc,all_sentence_id,all_pred_labels = evaluate_test()



              precision    recall  f1-score   support

           0       0.20      0.69      0.30       341
           1       0.95      0.70      0.81      3191

    accuracy                           0.70      3532
   macro avg       0.58      0.69      0.56      3532
weighted avg       0.88      0.70      0.76      3532

accuracy = 0.70


In [ ]:
pred_labels = list(le.inverse_transform(all_pred_labels))

In [ ]:
from collections import OrderedDict
dict_index_y_value = {}

for index,y_value in zip(all_sentence_id,pred_labels):
    dict_index_y_value[index] = y_value

od = OrderedDict(sorted(dict_index_y_value.items()))

sorted_y_predicts = []
for k,v in od.items():
    sorted_y_predicts.append(v)



In [ ]:
df_test['index_col'] = df_test.index


In [ ]:
test_tweet_id = list(df_test['index_col'].values)
test_ID = list(df_test['index_col'].values)


test_final = pd.DataFrame(zip(test_tweet_id,sorted_y_predicts,test_ID),columns=['index_col',task,'index_col'])

if task == 'task1':
    test_final.to_csv("submission_DE_A_HASOCOne_german_task_a_HASOC2019_GermEval2018.csv",index=False)

if task == 'task2':
    test_final.to_csv("submission_DE_B.csv",index=False)



In [ ]:
del model
import gc
gc.collect()



22